### Table of contents
[Predefined functions](#funcs)

[Parameters, filenames and directories](#params)

[Reading maps and catalogs](#read)

[SpecZ outlier filtering](#specZ)

[PhotoZ outlier filtering](#photoZ)

[ActivMap filtering](#activMap)

In [18]:
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from func_maps import readMaps
from func_stat import *
from func_visualizations import *
from os.path import join
from ast import literal_eval

### Parameters, filenames and directories
<a id='params'>#params</a>

In [12]:
### Files and directories ###
dirSOM=r'D:\Sources\COSMOS_photoZ\COSMOS_v3.0\SOM\forArticle'
nameOfExperiment='ex017'
dirMaps=join(dirSOM,nameOfExperiment,'maps')
dirPlots=join(dirSOM,nameOfExperiment,'plots')
dirDatasets=join(dirSOM,nameOfExperiment,'datasets')
mapsAllKeys=['mapsTrainOnTrain','mapsTestOnTrain','mapsDeimosOnTrain','mapsRunOnTrain',
             'mapsTrainOnRun','mapsTestOnRun','mapsDeimosOnRun','mapsRunOnRun',
             'mapsTrainOnRunBig','mapsTestOnRunBig','mapsDeimosOnRunBig','mapsRunOnRunBig']

In [24]:
datasetNames={'Train':r'05_COSMOS_SpectrZ_Zlim_1p2_SOMinfo.csv',
              'Test':r'05_COSMOS_SpectrZ_Zlim_1p2_Test_SOMinfo.csv',
              'Deimos':r'04_COSMOS2015_run_WorkBands_SOMinfo.csv',
              'Run':r'04_COSMOS_DEIMOS_QfFilter_SOMinfo.csv'}

In [14]:
idCol='Seq'
specZ='specZ'
photoZ_ML='photoZ_ML'
photoZ_SED='photoZ_SED'
residML='resid_ML'
residSED='resid_SED'
residML_SED='residML_SED'
quantErr='quantErr'

In [15]:
### Format for saving images ###
imSaveFormat='.png'
#imSaveFormat='.eps'

### Reading datasets
<a id='read'>#read</a>

In [26]:
datasets={}
cellIDs=['cellID_TrainSOM','cellID_RunSOM','cellID_RunBigSOM']
for key,val in datasetNames.items():
    datasets[key]=pd.read_csv(join(dirDatasets,val))
    for cellID in cellIDs:
        datasets[key][cellID]=[literal_eval(x) for x in datasets[key][cellID]]

### Stat table structure declaration
<a id='statDeclare'>#statDeclare</a>
<p>Since we have a bit too many statistical tables, they are stashed in one dictionary. It's structure is as follows:</p>
    <br>Dictionary statAll has keys which correspond to datasets.
    <br>For each key we have a dictionary which keys correspond to stat table names. We have: 
    <br>- 'basic' table, which corresponds to different cases of cleaning (None, SpecZ, PhotoZ, activMap and their combinations);
    <br>- 'magBin' table, which describes statistics within magnitude bins;
    <br>- 'oultCoeff' for specZ, SED and ML photoZ, which describe statistics within each bin of outlier coefficients;
    <br>- 'activMap' table, which describes statistics for different occupations of cells;
    <br>- probably something else.

In [27]:
statAll={name:{} for name in datasetNames.keys()}

### SpecZ outlier filtering
<a id='specZ'>#specZ</a>

In [33]:
datasets['Run'].columns

Index(['Seq', 'photoZ_ML', 'RAJ2000', 'DEJ2000', 'Ksmagap3', 'Ymagap3',
       'Hmagap3', 'Jmagap3', 'Bmagap3', 'Vmagap3', 'ipmagap3', 'rmagap3',
       'umagap3', 'zppmagap3', 'IB574ap3', 'photoZ_SED', 'specZ', 'Qf', 'Q',
       'sc_Ksmagap3', 'sc_Ymagap3', 'sc_Hmagap3', 'sc_Jmagap3', 'sc_Bmagap3',
       'sc_Vmagap3', 'sc_ipmagap3', 'sc_rmagap3', 'sc_umagap3', 'sc_zppmagap3',
       'resid_ML', 'resid_SED', 'residML_SED', 'w_sc_Ksmagap3', 'w_sc_Ymagap3',
       'w_sc_Hmagap3', 'w_sc_Jmagap3', 'w_sc_Bmagap3', 'w_sc_Vmagap3',
       'w_sc_ipmagap3', 'w_sc_rmagap3', 'w_sc_umagap3', 'w_sc_zppmagap3',
       'quantErr', 'cellID_TrainSOM', 'cellID_RunSOM', 'cellID_RunBigSOM',
       'specZ_outlCoeff', 'photoZ_ML_outlCoeff', 'photoZ_SED_outlCoeff',
       'quantErr_outlCoeff'],
      dtype='object')

In [31]:
### Calculate statistics for objects with different outlCoeff ###
cases=[specZ,photoZ_ML,photoZ_SED]
residCols=[residML,residSED,residML_SED]
bins=np.linspace(-7, 6, 26)
for name in statAll.keys():
    for c in cases:
        print(name)
        statAll[name]['outlCoeff_'+c]=statByBins(datasets[name],binCol=c+'_outlCoeff',residCols=residCols,bins=bins)

Train
Train
Train
Test
Test
Test
Deimos


KeyError: 'specZ_outlCoeff'

In [ ]:
### Plot statistics for different outlCoeff ###
